養成每日寫data science code的習慣！
改用慢慢實作專案累積知識以及實作能力，以NLP、TS、CV為主！

## 目標: 增加更多 Prototype 以及各應用實作能力！

### 本次目標
> 學習時間序列與機器學習建模，透過Kaggle課程，並擷取部分code。

### Trend

In [21]:
# 記得這是 statsmodels 14.0 以上的版本才有的功能！
from statsmodels.tsa.deterministic import DeterministicProcess
import numpy as np
import pandas as pd
from pandas import date_range
import matplotlib.pyplot as plt
%matplotlib inline


# 透過 pd 的函數建立日期序列
index = date_range("2000-1-1", freq="M", periods=240)
index

DatetimeIndex(['2000-01-31', '2000-02-29', '2000-03-31', '2000-04-30',
               '2000-05-31', '2000-06-30', '2000-07-31', '2000-08-31',
               '2000-09-30', '2000-10-31',
               ...
               '2019-03-31', '2019-04-30', '2019-05-31', '2019-06-30',
               '2019-07-31', '2019-08-31', '2019-09-30', '2019-10-31',
               '2019-11-30', '2019-12-31'],
              dtype='datetime64[ns]', length=240, freq='M')

In [22]:
# 透過此序列去建立 time-step 特徵，根據 DeterministicProcess

dp = DeterministicProcess(index=index, order=2, constant=True)
dp.in_sample()

,const,trend,trend_squared
2000-01-31,1.0,1.0,1.0
2000-02-29,1.0,2.0,4.0
2000-03-31,1.0,3.0,9.0
2000-04-30,1.0,4.0,16.0
2000-05-31,1.0,5.0,25.0
...,...,...,...
2019-08-31,1.0,236.0,55696.0
2019-09-30,1.0,237.0,56169.0
2019-10-31,1.0,238.0,56644.0
2019-11-30,1.0,239.0,57121.0


In [23]:
# 可以透過另外的 function 建立往後的特徵
# steps代表時間區間，因為建立的樣本是以一個月為單位，以下就是產生10個月的樣本

dp.out_of_sample(steps=10)

,const,trend,trend_squared
2020-01-31,1.0,241.0,58081.0
2020-02-29,1.0,242.0,58564.0
2020-03-31,1.0,243.0,59049.0
2020-04-30,1.0,244.0,59536.0
2020-05-31,1.0,245.0,60025.0
2020-06-30,1.0,246.0,60516.0
2020-07-31,1.0,247.0,61009.0
2020-08-31,1.0,248.0,61504.0
2020-09-30,1.0,249.0,62001.0
2020-10-31,1.0,250.0,62500.0


> 透過`DeterministicProcessd`可以簡單的建立各種 `time-step` 特徵，常可以用來去 fit trend，搭配一般無法學習到時間趨勢的機器學習模型，如LR、RF等等，就可以幫助捕捉 `Trend` Pattern。

### Seasonality
1. Indicators
2. Fourier features

In [24]:
# Fourier features的計算邏輯


def fourier_features(index, freq, order):
    """
    
    """
    time = np.arange(len(index), dtype=np.float32)
    k = 2 * np.pi * (1 / freq) * time
    features = {}
    for i in range(1, order+1):
        features.update({
            f"sin_{freq}_{i}": np.sin(i*k),
            f"cos_{freq}_{i}": np.cos(i*k)
        })
    return pd.DataFrame(features, index=index)

In [25]:
fourier_features(index, freq=365.25, order=4)

,sin_365.25_1,cos_365.25_1,sin_365.25_2,cos_365.25_2,sin_365.25_3,cos_365.25_3,sin_365.25_4,cos_365.25_4
2000-01-31,0.000000,1.000000,0.000000,1.000000,0.000000,1.000000,0.000000,1.000000
2000-02-29,0.017202,0.999852,0.034398,0.999408,0.051584,0.998669,0.068755,0.997634
2000-03-31,0.034398,0.999408,0.068755,0.997634,0.103031,0.994678,0.137185,0.990545
2000-04-30,0.051584,0.998669,0.103031,0.994678,0.154204,0.988039,0.204966,0.978769
2000-05-31,0.068755,0.997634,0.137185,0.990545,0.204966,0.978769,0.271777,0.962360
...,...,...,...,...,...,...,...,...
2019-08-31,-0.783934,-0.620844,0.973402,-0.229105,-0.424729,0.905321,-0.446022,-0.895022
2019-09-30,-0.794497,-0.607268,0.964945,-0.262452,-0.377463,0.926025,-0.506503,-0.862238
2019-10-31,-0.804826,-0.593511,0.955346,-0.295489,-0.329192,0.944263,-0.564588,-0.825373
2019-11-30,-0.814916,-0.579579,0.944617,-0.328176,-0.280045,0.959987,-0.620001,-0.784601


In [26]:
from pathlib import Path              # 用於整合 windows/unix 系統之間的路徑
from warnings import simplefilter     # 忽略警告訊息使用

import seaborn as sns
from sklearn.linear_model import LinearRegression
from statsmodels.tsa.deterministic import CalendarFourier, DeterministicProcess

In [27]:
Path('.')

WindowsPath('.')

In [28]:
simplefilter('ignore')


# 設定 matplotlib 
plt.style.use('seaborn-whitegrid')
plt.rc('figure', autolayout=True, figsize=(11, 5))  # 針對某一個元素進行設定
plt.rc('axes', labelweight='bold', labelsize='large', titleweight="bold", titlesize=16, titlepad=10)
plot_params = dict(
    color="0.75",
    style=".-",
    markeredgecolor="0.25",
    markerfacecolor="0.25",
    legend=False,
)

In [29]:
pd.Timedelta("1Y")

Timedelta('365 days 05:49:12')

In [30]:
pd.Timedelta("1D")

Timedelta('1 days 00:00:00')

In [31]:
pd.Timedelta("1Y") / pd.Timedelta("1D")

365.2425

In [32]:
def seasonal_plot(X, y, period, freq, ax=None):
    if ax is None:
        _, ax = plt.subplots()
        
    palette = sns.color_palette("hust", n_colors=X[period].nunique())  # 根據不同時間區間產生不同顏色
    ax = sns.lineplot(  # Draw a line plot with possibility of several semantic groupings.
        x=freq,
        y=y,
        hue=period,
        data=X,
        ci=False,
        ax=ax,
        palette=palette,
        legend=False
    )
    ax.set_title(f'Seasonal Plot({perid}/{freq})')
    for line, name in zip(ax.lines, X[period].unique()):
        y_ = line.get_ydata()[-1]
        ax.annotate( # Annotate the point xy with text text.
            name,
            xy=(1, y_),
            xytext=(6, 0),
            color=line.get_color(),
            xycoords=ax.get_yaxis_transform(),
            textcoords="offset points",
            size=14,
            va="center",
        )
    return ax


def plot_periodogram(ts, detrend='linear', ax=None):
    from scipy.signal import periodogram
    fs = pd.Timedelta('1Y') / pd.Timedelta("1D")
    freqencies, spectrum = periodogram(
        ts,
        fs=fs,
        detrend=detrend,
        window='boxcar',
        scaling='spectrum'
    )
    if ax is None:
        _, ax = plt.subplots()
    
    ax.step(freqencies, spectrum, color="purple") # Make a step plot.
    ax.set_xscale("log")                         # 為了尺度: The axis scale type to apply.
    ax.set_xticks([1, 2, 4, 6, 12, 26, 52, 104])  # 
    ax.set_xticklabels(
        [
            "Annual (1)",
            "Semiannual (2)",
            "Quarterly (4)",
            "Bimonthly (6)",
            "Monthly (12)",
            "Biweekly (26)",
            "Weekly (52)",
            "Semiweekly (104)",
        ],
        rotations=30
    )
    ax.ticklabel_format(axis='y', style='sci', scilimits=(0, 0))
    ax.set_ylabel('Variance')
    ax.set_title('Periodogram')
    return ax

In [33]:
from statsmodels.tsa.deterministic import CalendarFourier, DeterministicProcess

# Fourier series deterministic terms based on calendar time
# ref: https://www.statsmodels.org/dev/generated/statsmodels.tsa.deterministic.CalendarFourier.html
fourier = CalendarFourier(freq="A", order=10)  # 10 sin/cos pairs for "A"nnual seasonality

dp = DeterministicProcess(
    index=index,
    constant=True,               # dummy feature for bias (y-intercept)
    order=1,                     # trend (order 1 means linear)
    seasonal=True,               # weekly seasonality (indicators)
    additional_terms=[fourier],  # annual seasonality (fourier)
    drop=True,                   # drop terms to avoid collinearity
)

X = dp.in_sample()  # create features for dates in tunnel.index

In [34]:
print(X.shape)
X

(240, 25)


,const,trend,"s(2,12)","s(3,12)","s(4,12)","s(5,12)","s(6,12)","s(7,12)","s(8,12)","s(9,12)",...,"sin(2,freq=A-DEC)","cos(2,freq=A-DEC)","sin(3,freq=A-DEC)","cos(3,freq=A-DEC)","sin(4,freq=A-DEC)","cos(4,freq=A-DEC)","sin(5,freq=A-DEC)","cos(5,freq=A-DEC)","sin(6,freq=A-DEC)","cos(6,freq=A-DEC)"
2000-01-31,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.857315,0.514793,0.999668,0.025748,0.882679,-0.469977,0.536696,-0.843776,0.051479,-0.998674
2000-02-29,1.0,2.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.898292,-0.439400,0.102821,-0.994700,-0.789418,-0.613856,-0.938710,0.344707,-0.204552,0.978856
2000-03-31,1.0,3.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.051479,-0.998674,-0.997018,-0.077175,-0.102821,0.994700,0.991723,0.128398,0.153891,-0.988088
2000-04-30,1.0,4.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,-0.829677,-0.558244,-0.102821,0.994700,0.926324,-0.376728,-0.767880,-0.640593,-0.204552,0.978856
2000-05-31,1.0,5.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,-0.890617,0.454755,0.997018,0.077175,-0.810025,-0.586396,0.384665,0.923056,0.153891,-0.988088
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2019-08-31,1.0,236.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,...,0.888057,-0.459733,-0.068802,0.997630,-0.816538,-0.577292,0.917584,-0.397543,-0.137279,0.990532
2019-09-30,1.0,237.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,...,0.060213,-0.998186,0.995919,0.090252,-0.120208,0.992749,-0.988678,-0.150055,0.179767,-0.983709
2019-10-31,1.0,238.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,-0.845249,-0.534373,0.060213,-0.998186,0.903356,-0.428892,0.811539,0.584298,-0.120208,0.992749
2019-11-30,1.0,239.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,-0.891981,0.452072,-0.996659,-0.081676,-0.806480,-0.591261,-0.377708,-0.925925,0.162807,-0.986658


In [35]:
fourier

Fourier(freq=A-DEC, order=10) at 0x1e3f336b128

In [36]:
# y = # 時間序列標籤

# model = LinearRegression(fit_intercept=False)
# _ = model.fit(X, y)

# y_pred = pd.Series(model.predict(X), index=y.index)
# X_fore = dp.out_of_sample(steps=90)
# y_fore = pd.Series(model.predict(X_fore), index=X_fore.index)

# ax = y.plot(color='0.25', style='.', title="Tunnel Traffic - Seasonal Forecast")
# ax = y_pred.plot(ax=ax, label="Seasonal")
# ax = y_fore.plot(ax=ax, label="Seasonal Forecast", color='C3')
# _ = ax.legend()

### Time Series as Features
- 序列相依。
- 與季節性不同，與前幾期的時間相關，但與絕對日期不相關(主要差異)。

In [39]:
import pandas as pd

# 簡單使用上面的資料去建立 lag features
df = pd.DataFrame({
    'y': X['trend'],
    'y_lag_1': X['trend'].shift(1),
    'y_lag_2': X['trend'].shift(2)
})

print(df.shape)
df.head()

(240, 3)


,y,y_lag_1,y_lag_2
2000-01-31,1.0,NaN,NaN
2000-02-29,2.0,1.0,NaN
2000-03-31,3.0,2.0,1.0
2000-04-30,4.0,3.0,2.0
2000-05-31,5.0,4.0,3.0


#### Lag plots
- A lag plot of a time series shows its values plotted against its lags. Serial dependence in a time series will often become apparent by looking at a lag plot.
- 常用的是 autocorrelation。
- Plotting the partial autocorrelation can help you choose which lag features to use. 
- The correlogram is for lag features essentially what the periodogram is for Fourier features.
- Finally, we need to be mindful that autocorrelation and partial autocorrelation are measures of linear dependence.

---

all above from [kaggle tutorials](https://www.kaggle.com/code/ryanholbrook/time-series-as-features#What-is-Serial-Dependence?)